In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
root = "final.csv"
batch_size = 256
epochs = 200
learning_rate = 1e-3
seed = 42

In [3]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
class CsvDataset(Dataset):
    def __init__(self, path):
        super(CsvDataset).__init__()
        csv = pd.read_csv(path)
        csv_np = csv.to_numpy()
        for i in [1, 3, 4, 5, 6, 7, 8]:
            wordset = {word: idx for idx, word in enumerate(np.unique(csv_np[:,i]))}
            for row in range(len(csv_np)):
                csv_np[row][i] = wordset[csv_np[row][i]]
        self.ints = torch.from_numpy(np.array(csv_np[:,[1,3,4,5,6,7,8]], dtype="int"))
        self.floats = torch.from_numpy(np.array(csv_np[:,[2,9,10,11]], dtype="float")).float()
        self.target = torch.from_numpy(np.array(csv_np[:,[12]], dtype="float")).float()
        
        
    def __getitem__(self,idx):
        return self.ints[idx],self.floats[idx], self.target[idx]
    
    
    def __len__(self):
        return len(self.ints)

In [5]:
data_length = len(pd.read_csv(root))
train_length = int(data_length * 0.6)
test_length = int(data_length * 0.2)
val_length = data_length - train_length - test_length

train_dataset = CsvDataset(root)
train_dataset, test_dataset = random_split(train_dataset, [train_length, test_length+val_length])
test_dataset, val_dataset = random_split(test_dataset, [test_length, val_length])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [6]:
class CsvNet(nn.Module):
    def __init__(self):
        super(CsvNet, self).__init__()
        self.emb1 = torch.nn.Embedding(8, 20)
        self.emb2 = torch.nn.Embedding(11, 20)
        self.emb3 = torch.nn.Embedding(20, 20)
        self.emb4 = torch.nn.Embedding(4, 20)
        self.emb5 = torch.nn.Embedding(4, 20)
        self.emb6 = torch.nn.Embedding(4, 20)
        self.emb7 = torch.nn.Embedding(7, 20)
        self.act = nn.ReLU()
        self.fc = nn.Linear(4, 80)
        self.bn = nn.BatchNorm1d(80)
        self.fc1 = nn.Linear(220, 8192)
        self.fc2 = nn.Linear(8192, 8192)
        self.fc3 = nn.Linear(8192, 4096)
        self.fc4 = nn.Linear(4096, 2048)
        self.fc5 = nn.Linear(2048, 1024)
        self.fc6 = nn.Linear(1024, 1)
        self.bn1 = nn.BatchNorm1d(8192)
        self.bn2 = nn.BatchNorm1d(8192)
        self.bn3 = nn.BatchNorm1d(4096)
        self.bn4 = nn.BatchNorm1d(2048)
        self.bn5 = nn.BatchNorm1d(1024)
        self.dropout = nn.Dropout()
    
    def forward(self, x, y):
        x1 = self.emb1(x[:,0])
        x2 = self.emb2(x[:,1])
        x3 = self.emb3(x[:,2])
        x4 = self.emb4(x[:,3])
        x5 = self.emb5(x[:,4])
        x6 = self.emb6(x[:,5])
        x7 = self.emb7(x[:,6])
        y = self.bn(self.fc(y))
        x = torch.cat((x1, x2, x3, x4, x5, x6, x7, y), dim=1)
        x = self.dropout(self.act(self.bn1(self.fc1(x))))
        x = self.dropout(self.act(self.bn2(self.fc2(x))))
        x = self.dropout(self.act(self.bn3(self.fc3(x))))
        x = self.dropout(self.act(self.bn4(self.fc4(x))))
        x = self.dropout(self.act(self.bn5(self.fc5(x))))
        return self.fc6(x)

In [7]:
model = CsvNet().to(device)

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
for epoch in range(epochs):
    model.train()
    criterion.train()
    
    avg_loss = 0

    for X1, X2, Y in train_loader:
        X1 = X1.to(device)
        X2 = X2.to(device)
        Y = Y.to(device)

        model.zero_grad()
        prediction = model(X1, X2)
        loss = torch.sqrt(criterion(prediction, Y)).to(device)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
    print(f'[Epoch: {epoch+1:>2}] Average loss: {avg_loss:.4f}, ', end='')
    
    model.eval()
    criterion.eval()
    with torch.no_grad():
        val_avg_loss = 0.
        for X1_val, X2_val, Y_val in val_loader:
            X1_val = X1_val.to(device)
            X2_val = X2_val.to(device)
            Y_val = Y_val.to(device)
            val_prediction = model(X1_val, X2_val)
            val_loss = torch.sqrt(criterion(val_prediction, Y_val)).to(device)
            val_avg_loss += val_loss / len(val_loader)
        
        print(f"val_loss: {val_avg_loss:.4f}")

[Epoch:  1] Average loss: 4519.5762, val_loss: 4559.9575
[Epoch:  2] Average loss: 4533.3750, val_loss: 4797.3760
[Epoch:  3] Average loss: 4581.8599, val_loss: 5060.5278
[Epoch:  4] Average loss: 4457.4434, val_loss: 5000.6460
[Epoch:  5] Average loss: 4464.5142, val_loss: 4442.4307
[Epoch:  6] Average loss: 4421.5005, val_loss: 4526.6196
[Epoch:  7] Average loss: 4505.9775, val_loss: 4913.3276
[Epoch:  8] Average loss: 4425.6211, val_loss: 4611.4053
[Epoch:  9] Average loss: 4365.6475, val_loss: 4826.8535
[Epoch: 10] Average loss: 4508.2744, val_loss: 4631.7588
[Epoch: 11] Average loss: 4360.5522, val_loss: 4894.0371
[Epoch: 12] Average loss: 4473.0420, val_loss: 4708.0571
[Epoch: 13] Average loss: 4661.7061, val_loss: 4432.7959
[Epoch: 14] Average loss: 4464.1636, val_loss: 4710.3647
[Epoch: 15] Average loss: 4347.0547, val_loss: 4440.3643
[Epoch: 16] Average loss: 4374.8901, val_loss: 4371.7251
[Epoch: 17] Average loss: 4155.9185, val_loss: 4577.7129
[Epoch: 18] Average loss: 4411.

[Epoch: 144] Average loss: 1178.8431, val_loss: 1144.0959
[Epoch: 145] Average loss: 1305.5814, val_loss: 1607.1671
[Epoch: 146] Average loss: 1039.2130, val_loss: 1204.0282
[Epoch: 147] Average loss: 1125.1207, val_loss: 1237.5701
[Epoch: 148] Average loss: 1125.2358, val_loss: 1091.0691
[Epoch: 149] Average loss: 1261.6077, val_loss: 1243.1740
[Epoch: 150] Average loss: 1274.4833, val_loss: 1242.8267
[Epoch: 151] Average loss: 1123.8521, val_loss: 1213.3269
[Epoch: 152] Average loss: 1402.2902, val_loss: 1582.1960
[Epoch: 153] Average loss: 1265.3231, val_loss: 1293.2488
[Epoch: 154] Average loss: 1301.5834, val_loss: 999.6271
[Epoch: 155] Average loss: 1126.7878, val_loss: 1504.4102
[Epoch: 156] Average loss: 1296.0389, val_loss: 1120.7988
[Epoch: 157] Average loss: 1438.3850, val_loss: 1321.7009
[Epoch: 158] Average loss: 1236.9622, val_loss: 1147.7062
[Epoch: 159] Average loss: 1112.3372, val_loss: 1014.3842
[Epoch: 160] Average loss: 1026.1965, val_loss: 1064.2285
[Epoch: 161] Av

In [9]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
with torch.no_grad():
    for X1_test, X2_test, Y_test in test_loader:
        X1_test = X1_test.to(device)
        X2_test = X2_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(X1_test, X2_test)
        mean = torch.mean(Y_test)
        ss_tot += torch.sum((Y_test - mean) ** 2)
        ss_res += torch.sum((Y_test - prediction) ** 2)
    
    accuracy = 1 - ss_res/ss_tot
    print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 86.28%
